In [1]:
! pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import pathway as pw
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('dataset.csv')

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# making a new column called occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

df[['Timestamp', 'SystemCodeNumber', 'OccupancyRate', 'QueueLength',
    'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv('demand_data.csv', index=False)

DemandScore = α ⋅ Occupancy  +  β ⋅ Queue  +  γ ⋅ Traffic  +  δ ⋅ SpecialDay  +  ϵ ⋅ VehicleWeight

In [4]:
# defining weights that user can adjust based on their business models

weights = {
    'occupancy': 0.5,
    'queue': 0.3,
    'traffic': 0.2,
    'special_day': 0.4,
    'vehicle': {
        'car': 1.0,
        'bike': 0.7,
        'truck': 1.3
    }
}

# converting the TrafficConfitionNearby categories into numbers
traffic_levels = {
    'low': 0.2,
    'medium': 0.5,
    'high':0.8
}

In [5]:
# setting up pathway schema
class DemandSchema(pw.Schema):
  Timestamp: str
  SystemCodeNumber: str
  OccupancyRate: float
  QueueLength: int
  TrafficConditionNearby: str
  IsSpecialDay: int
  VehicleType: str

# now loading stream
demand_stream = pw.demo.replay_csv('demand_data.csv', schema=DemandSchema, input_rate=1000)

In [6]:
# making a function to calculate demand

def calculate_demand(row):
    # converting traffic to numeric values
    traffic_score = (
        pw.if_else(row.TrafficConditionNearby == "low", 0.2,
        pw.if_else(row.TrafficConditionNearby == "medium", 0.5,
        0.8))  # default for 'high'
    )

    # vehicle type weights
    vehicle_weight = (
        pw.if_else(row.VehicleType == "car", 1.0,
        pw.if_else(row.VehicleType == "bike", 0.7,
        1.3))  # default for 'truck'
    )

    demand = (
        weights['occupancy'] * row.OccupancyRate +
        weights['queue'] * (row.QueueLength / 10) +
        weights['traffic'] * traffic_score +
        weights['special_day'] * row.IsSpecialDay +
        0.1 * vehicle_weight
    )
    return demand

# applying function to stream
demand_data = demand_stream.with_columns(
    demand_score=calculate_demand(pw.this)
)

In [7]:
base_price = 10

# clamp between 0.5x to 2x
demand_data_clamped = demand_data.with_columns(
    clamped_score=pw.if_else(
        pw.this.demand_score < 0, 0,
        pw.if_else(
            pw.this.demand_score > 1, 1,
            pw.this.demand_score
        )
    )
)

# calculating prices
model2_output = demand_data_clamped.with_columns(
    price=base_price * (1 + pw.this.clamped_score),
    time=pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S"),
    occupancy_rate=pw.this.OccupancyRate,
    queue_length=pw.this.QueueLength,
    vehicle_type=pw.this.VehicleType
)

preview_data = pw.debug.table_to_pandas(model2_output)
print(preview_data.head(3))

# pw.debug.show(model2_output, include_id=False)

                                       Timestamp SystemCodeNumber  \
^TADE6RX1CKYKMKVD2G9TB6G014  2016-10-05 09:30:00      BHMBCCMKT01   
^KQHWMZ4NRTWFDVBCZ1RGFJ6P3M  2016-10-04 14:57:00      BHMBCCMKT01   
^AF7P7GA60QWVJBM7T7M53WC778  2016-10-05 10:30:00      BHMBCCMKT01   

                             OccupancyRate  QueueLength  \
^TADE6RX1CKYKMKVD2G9TB6G014       0.143847            2   
^KQHWMZ4NRTWFDVBCZ1RGFJ6P3M       0.332756            3   
^AF7P7GA60QWVJBM7T7M53WC778       0.221837            4   

                            TrafficConditionNearby  IsSpecialDay VehicleType  \
^TADE6RX1CKYKMKVD2G9TB6G014                average             0         car   
^KQHWMZ4NRTWFDVBCZ1RGFJ6P3M                average             0        bike   
^AF7P7GA60QWVJBM7T7M53WC778                   high             0       truck   

                             demand_score  clamped_score      price  \
^TADE6RX1CKYKMKVD2G9TB6G014      0.391924       0.391924  13.919237   
^KQHWMZ4NRTWFDVBCZ1RGFJ

In [11]:
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import output_notebook
import pandas as pd

output_notebook()

static_data = pw.debug.table_to_pandas(model2_output)

static_data['hour'] = static_data['time'].dt.hour.astype(str)
available_hours = sorted(static_data['hour'].unique())
all_lots = sorted(static_data['SystemCodeNumber'].unique())

# creating a grid of prices
hourly_avg = (static_data.groupby(['SystemCodeNumber', 'hour'])['price']
              .mean()
              .unstack()
              .reindex(index=all_lots, columns=available_hours)
              .stack()
              .reset_index(name='price'))

source = ColumnDataSource({
    'hour_str': hourly_avg['hour'].astype(str),
    'SystemCodeNumber': hourly_avg['SystemCodeNumber'],
    'price': hourly_avg['price'],
    'hour': hourly_avg['hour']  # For tooltips
})

p = figure(
    width=900,
    height=400,
    title=f"Hourly Prices ({int(available_hours[0])}:00-{int(available_hours[-1])}:00)",
    x_range=available_hours,
    y_range=all_lots,
    tools="hover,save",
    toolbar_location="above",
    x_axis_label="Hour of Day",
    y_axis_label="Parking Lot"
)

# heatmap rectangles
p.rect(
    x='hour_str',
    y='SystemCodeNumber',
    width=0.9,
    height=0.9,
    source=source,
    fill_color=linear_cmap(
        field_name='price',
        palette=Viridis256,
        low=hourly_avg['price'].min(),
        high=hourly_avg['price'].max()
    ),
    line_color=None
)

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Lot", "@SystemCodeNumber"),
    ("Hour", "@hour:00"),
    ("Price", "@price{$0.00}")
]

show(p)

In [9]:
# viz 2. daily price trends

# grouping data by hour and parking lot
hourly_trends = static_data.groupby(['SystemCodeNumber', 'hour'])['price'].mean().unstack()

# only plotting 5 busiest lots
busiest_lots = static_data['SystemCodeNumber'].value_counts().index[:5]
p = figure(width=900, height=400, title="Daily Price Trends", x_range=(8,17))

for lot in busiest_lots:
    p.line(
        x=hourly_trends.columns,
        y=hourly_trends.loc[lot],
        legend_label=lot,
        line_width=2
    )

p.xaxis.axis_label = "Hour of Day"
p.yaxis.axis_label = "Price ($)"
p.legend.location = "top_left"
show(p)

In [10]:
# viz 3. price vs occupancy rate

from bokeh.models import (ColumnDataSource, LinearColorMapper,
                         ColorBar, HoverTool, NumeralTickFormatter)
from bokeh.palettes import Viridis256

# converting pathway data to pandas
raw_data = pw.debug.table_to_pandas(model2_output)
viz_data = pd.DataFrame({
    'occupancy_rate': raw_data['occupancy_rate'].astype(float),
    'price': raw_data['price'].astype(float),
    'time': pd.to_datetime(raw_data['time']),
    'lot_id': raw_data['SystemCodeNumber'].astype(str),
    'queue': raw_data['queue_length'].astype(int)
})

# extracting hour of day
viz_data['time_of_day'] = viz_data['time'].dt.hour

plot_data = ColumnDataSource({
    'occupancy_rate': viz_data['occupancy_rate'],
    'price': viz_data['price'],
    'time_of_day': viz_data['time_of_day'],
    'lot_id': viz_data['lot_id'],
    'queue': viz_data['queue']
})

# creating plot
p = figure(
    width=800,
    height=500,
    title="Price vs. Occupancy Rate",
    x_axis_label='Occupancy Rate (%)',
    y_axis_label='Price ($)',
    tools="hover,pan,wheel_zoom,box_select,reset",
    toolbar_location="below"
)

# adding scatter plot
color_mapper = LinearColorMapper(
    palette=Viridis256,
    low=viz_data['time_of_day'].min(),
    high=viz_data['time_of_day'].max()
)

p.scatter(
    x='occupancy_rate',
    y='price',
    size=8,
    source=plot_data,
    fill_color={'field': 'time_of_day', 'transform': color_mapper},
    line_color=None,
    alpha=0.7
)

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Lot", "@lot_id"),
    ("Hour", "@time_of_day:00"),
    ("Occupancy", "@occupancy_rate{0.0%}"),
    ("Price", "@price{$0.00}"),
    ("Queue", "@queue")
]

color_bar = ColorBar(
    color_mapper=color_mapper,
    label_standoff=12,
    title="Hour of Day"
)
p.add_layout(color_bar, 'right')

p.xaxis.formatter = NumeralTickFormatter(format="0%")
p.yaxis.formatter = NumeralTickFormatter(format="$0.00")

show(p)